대학, 학과, 교번, 성명, 풀네임, 약어, Scopus ID, Scopus ID 갯수, PNU ID, 직급, 이메일, 임용일자, 퇴직일자

In [22]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
from openpyxl import load_workbook

pd.options.display.float_format = '{:.0f}'.format
pd.options.display.max_columns = None

jeunyim_list = pd.read_excel("연구자명단/교무과제공_전임교원 명단_임용일자추가(2020.4.27.기준).xlsx")
giegum_list = pd.read_excel("연구자명단/교무과제공_기금교수 명단_임용일자추가(2020.4.20.현재기준).xlsx")
profess_list = pd.read_excel("연구자명단/도서관제공_교수급 연구자명단(원본).xlsx")
pip_all = pd.read_excel("PIP_ALL.xlsx")

jeunyim_cnt = len(jeunyim_list)
giegum_cnt = len(giegum_list)

Researcher_temp_list = df(columns=["대학","학과","성명","영문풀네임","영문약어","직급","교번","PNU_ID","SCOPUS_ID"])
Researcher_final_list = df(columns=["대학","학과","성명","영문풀네임","영문약어","직급","교번","PNU_ID","SCOPUS_ID"])

# 전임교원 명단 추가
for pro in range(0, jeunyim_cnt):
    res_number = jeunyim_list.loc[pro][5] #교번은 유일
    if res_number in profess_list['교번'].tolist():
        index_num = profess_list['교번'].tolist().index(res_number)
    else:
        index_num=40
    
    Researcher_temp_list.loc[pro] = [
        jeunyim_list.loc[pro][1], #대학
        jeunyim_list.loc[pro][2], #학과
        jeunyim_list.loc[pro][4], #성명
        profess_list.loc[index_num][4], #영문풀네임 from 도서관
        profess_list.loc[index_num][5], #영문약어 from 도서관
        jeunyim_list.loc[pro][3], #직급
        jeunyim_list.loc[pro][5], #교번
        profess_list.loc[index_num][0], #PNU_ID from 도서관
        "NONE"#jeunyim_list.loc[pro][0], #SCOPUS_ID
    ]
    
# 기금교원 명단 추가
for pro in range(0,giegum_cnt):
    res_number = giegum_list.loc[pro][4] #교번은 유일
    if res_number in profess_list['교번'].tolist():
        index_num = profess_list['교번'].tolist().index(res_number)
    else:
        index_num=40
    
    Researcher_temp_list.loc[jeunyim_cnt+pro] = [
        giegum_list.loc[pro][1], #대학
        giegum_list.loc[pro][2], #학과
        giegum_list.loc[pro][5], #성명
        profess_list.loc[index_num][4], #영문풀네임 from 도서관
        profess_list.loc[index_num][5], #영문약어 from 도서관
        giegum_list.loc[pro][3], #직급
        giegum_list.loc[pro][4], #교번
        profess_list.loc[index_num][0], #PNU_ID from 도서관
        "NONE"#jeunyim_list.loc[pro][0], #SCOPUS_ID
    ]
    
 
#print(Researcher_temp_list.shape)
#Researcher_temp_list
#Researcher_total_list.to_excel("researcher_table.xlsx")
# 학과명별로 재분류
pnu_department_list = Researcher_temp_list["학과"].unique()
tmp=[]
for department in pnu_department_list:
    tmp.append(Researcher_temp_list.groupby('학과').get_group(department).sort_values('성명', ascending=True))

for dep in range(0,len(tmp)):
    Researcher_final_list = pd.concat([Researcher_final_list,tmp[dep]])
    
#Researcher_final_list.to_excel("저자종합명단.xlsx")
Researcher_list_v1 = Researcher_final_list.reset_index(drop=True)
total_prf_cnt = len(Researcher_list_v1)

#Researcher_list_v1


In [23]:
Researcher_list_v3 = df(columns=["대학","학과","성명","교번","풀네임","약어",
                                 "S_ID_1",
                                 "S_ID_2",
                                 "S_ID_Num",
                                 "P_docs",
                                 "P_docs_5",
                                 "S_docs",
                                 "S_docs_5",
                                 "PNU_ID","연구자번호","직급","이메일","임용일","퇴직일"])


In [24]:
for pro in range(0,total_prf_cnt):
    
    kyo_num = Researcher_list_v1.loc[pro][6]
    email=None
    app_date=None
    ret_date=None
    
    #print(kyo_num)
    if kyo_num in profess_list['교번'].tolist():
        email=profess_list['email'].loc[profess_list['교번'].tolist().index(kyo_num)]
    if kyo_num in pip_all['교직원번호'].tolist():
        app_date=pip_all['임용일자'].loc[pip_all['교직원번호'].tolist().index(kyo_num)]
        ret_date=pip_all['퇴직일자'].loc[pip_all['교직원번호'].tolist().index(kyo_num)]
        res_num=pip_all['연구자번호'].loc[pip_all['교직원번호'].tolist().index(kyo_num)]
    
    Researcher_list_v3.loc[pro] = [
        Researcher_list_v1.loc[pro][0], # 대학
        Researcher_list_v1.loc[pro][1], # 학과
        Researcher_list_v1.loc[pro][2], # 성명
        kyo_num, # 교번
        Researcher_list_v1.loc[pro][3], # 풀네임
        Researcher_list_v1.loc[pro][4], # 약어
        
        None,#Researcher_list_v1.loc[pro][], # Scopus id-1
        None,#Researcher_list_v1.loc[pro][], # Scopus id-2
        None,#Researcher_list_v1.loc[pro][], # Scopus id number
        None,#Researcher_list_v1.loc[pro][], # Scopus pip_documents
        None,#Researcher_list_v1.loc[pro][], # Scopus pip_documents_5
        None,#Researcher_list_v1.loc[pro][], # Scopus scopus_documents
        None,#Researcher_list_v1.loc[pro][], # Scopus scopus_documents_5
        
        Researcher_list_v1.loc[pro][7], # pnu id
        res_num,
        Researcher_list_v1.loc[pro][5], # 직급
        email, # 이메일
        app_date, # 임용일
        ret_date, # 퇴직일
    ]
                        
#Researcher_list_v3.to_excel("저자종합명단.xlsx")

In [25]:
Researcher_list_v3

,대학,학과,성명,교번,풀네임,약어,S_ID_1,S_ID_2,S_ID_Num,P_docs,P_docs_5,S_docs,S_docs_5,PNU_ID,연구자번호,직급,이메일,임용일,퇴직일
0,공과대학,전기공학과,KANDASAMY PRABAKAR,111424,KANDASAMY PRABAKAR,"Prabakar, K",None,None,None,None,None,None,None,1100,10185452,교수,kprabakar@gmail.com,20080307,NaN
1,공과대학,전기공학과,권영안,110435,"Kwon, Young Ahn","Kwon, YA",None,None,None,None,None,None,None,1085,10052279,교수,yakwon@pusan.ac.kr,19870311,NaN
2,공과대학,전기공학과,김성신,110833,"Kim, Sungshin","Kim, S",None,None,None,None,None,None,None,1086,10054869,교수,sskim@pusan.ac.kr,19980325,NaN
3,공과대학,전기공학과,김욱,111573,"Kim, Wook","Kim, W",None,None,None,None,None,None,None,1087,11003384,부교수,NaN,20110901,NaN
4,공과대학,전기공학과,김장목,110901,"Kim, Jang-Mok","Kim, JM",None,None,None,None,None,None,None,1088,10124698,교수,jmok@pusan.ac.kr,20010301,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,부속연구소,점필재연구소,신상필,111803,NaN,NaN,None,None,None,None,None,None,None,895,10044664,조교수,None,20170831,NaN
1341,부속연구소,점필재연구소,임상석,111958,NaN,NaN,None,None,None,None,None,None,None,895,nan,조교수,None,20190801,NaN
1342,치의학전문대학원,치의생명과학과,김형식,111867,NaN,NaN,None,None,None,None,None,None,None,895,nan,조교수,None,20180901,NaN
1343,공과대학,고분자공학과,김채빈,111919,NaN,NaN,None,None,None,None,None,None,None,895,11647857,조교수,None,20190920,NaN
